<a href="https://colab.research.google.com/github/ugywjd/pancancer/blob/master/phj_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
exp = pd.read_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/CCLE_expression.csv')#cellline_RNA_expression
c_TCGA = pd.read_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/CCLE_TCGA.csv') #cellline_TCGA annotation
gdsc_fit = pd.read_excel('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/GDSC2_fitted_dose_response_24Jul22.xlsx') #gdsc2 data

In [4]:
#결측치 제거
c_TCGA = c_TCGA.dropna(axis = 0)
exp = exp.drop(labels = ['Unnamed: 0'], axis=1)
exp_index = exp['DepMap_ID']
#cellline_TCGA.isnull().sum()

In [ ]:
exp

In [6]:
#GBM, LGG cellline만 가져오기
#cellline_TCGA = cellline_TCGA[(cellline_TCGA['TCGA']=='GBM')|(cellline_TCGA['TCGA']=='LGG')]

In [ ]:
#cellline, gdsc2 합치기
cell_gdsc = pd.merge(c_TCGA, gdsc_fit, left_on='CCLE', right_on='CELL_LINE_NAME')
cell_gdsc

In [8]:
### drug name normalization
drug_ref_1 = pd.read_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/drug_response_predict/raw_data/DrugCorrection1.csv')
drug_ref_2 = pd.read_excel('/content/drive/MyDrive/Hong_Lab/21. Depmap/drug_response_predict/raw_data/bioinfo16_supplementary_tables.xlsx', sheet_name=1, header=2).drop('DrugBank ID', axis=1)
drug_name = cell_gdsc.merge(drug_ref_1, how = 'left', left_on = 'DRUG_NAME', right_on = 'OldName').drop('OldName', axis=1)
drug_name = cell_gdsc.merge(drug_ref_2, how = 'left', left_on = 'DRUG_NAME', right_on = 'Recorded Name from TCGA').drop('Recorded Name from TCGA', axis=1)

In [9]:
drug_Tzm = drug_name[(drug_name['DRUG_NAME']=='Temozolomide')] #cellline, gdsc랑 매칭되는 Temozolomide만 가져오기

In [10]:
drug_Tzm.to_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/cell_Tzm.csv',index=False)#cellline, gdsc랑 매칭되는 Temozolomide 저장

In [11]:
# z-score값 기준으로 resistant, sensitive cell line 나누기
drug_Tzm['measure_of_response'] = drug_Tzm['Z_SCORE']
drug_Tzm = drug_Tzm[(drug_Tzm['measure_of_response'] >= 2 )|(drug_Tzm['measure_of_response'] <= -2)]

drug_Tzm.loc[drug_Tzm['measure_of_response'] > 2, 'measure_of_response'] = 'Resistant'
drug_Tzm.loc[drug_Tzm['Z_SCORE'] < -2, 'measure_of_response'] = 'Sensitive'
drug_Tzm

<ipython-input-11-3b00dfb3c7c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drug_Tzm['measure_of_response'] = drug_Tzm['Z_SCORE']
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,DepMap_ID,CELLLINE,CCLE,TCGA,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,...,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE,Standard drug name,measure_of_response
3363,ACH-000350,COLO678_LARGE_INTESTINE,COLO-678,COADREAD,GDSC2,401,19085138,910689,COLO-678,SIDM00957,...,1046,Y,0.030016,30.0,9.624756,0.973549,0.068474,2.578816,Temozolomide,Resistant
9638,ACH-000638,NCIH441_LUNG,NCI-H441,LUAD,GDSC2,401,19054580,908460,NCI-H441,SIDM00925,...,1046,Y,0.030016,30.0,8.891349,0.965973,0.077245,2.015987,Temozolomide,Resistant
20149,ACH-000113,OCIAML2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,OCI-AML2,LAML,GDSC2,401,19099928,910947,OCI-AML2,SIDM00446,...,1046,Y,0.030016,30.0,3.647879,0.942202,0.137500,-2.007937,Temozolomide,Sensitive
21621,ACH-000222,ASPC1_PANCREAS,AsPC-1,PAAD,GDSC2,401,19087466,910702,AsPC-1,SIDM00899,...,1046,Y,0.030016,30.0,9.811507,0.969569,0.051331,2.722131,Temozolomide,Resistant
23858,ACH-000308,EFO21_OVARY,EFO-21,OV,GDSC2,401,19101060,911905,EFO-21,SIDM01052,...,1046,Y,0.030016,30.0,9.203067,0.970885,0.055270,2.255204,Temozolomide,Resistant
25708,ACH-000365,SUDHL4_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,SU-DHL-4,DLBC,GDSC2,401,19176070,1331035,SU-DHL-4,SIDM00405,...,1046,Y,0.030016,30.0,3.337333,0.917222,0.242766,-2.246254,Temozolomide,Sensitive
29685,ACH-000648,NCIH28_PLEURA,NCI-H28,MESO,GDSC2,401,19056492,908470,NCI-H28,SIDM00720,...,1046,Y,0.030016,30.0,8.998074,0.967058,0.077005,2.097890,Temozolomide,Resistant
49703,ACH-000759,MDAMB175VII_BREAST,MDA-MB-175-VII,BRCA,GDSC2,401,19041772,908120,MDA-MB-175-VII,SIDM00633,...,1046,Y,0.029297,30.0,9.337230,0.979750,0.067217,2.358164,Temozolomide,Resistant
56009,ACH-000346,JVM3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,JVM-3,DLBC,GDSC2,401,19029322,907270,JVM-3,SIDM01012,...,1046,Y,0.030016,30.0,2.247410,0.785896,0.093411,-3.082679,Temozolomide,Sensitive
59027,ACH-000656,SUDHL8_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,SU-DHL-8,DLBC,GDSC2,401,19176797,1331038,SU-DHL-8,SIDM00423,...,1046,Y,0.030016,30.0,3.203253,0.920825,0.092770,-2.349150,Temozolomide,Sensitive


In [12]:
drug_Tzm.to_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/cell_Tzm.csv',index=False)#cellline, gdsc랑 매칭되는 Temozolomide 저장

In [13]:
drug_Tzm

,DepMap_ID,CELLLINE,CCLE,TCGA,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,...,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE,Standard drug name,measure_of_response
3363,ACH-000350,COLO678_LARGE_INTESTINE,COLO-678,COADREAD,GDSC2,401,19085138,910689,COLO-678,SIDM00957,...,1046,Y,0.030016,30.0,9.624756,0.973549,0.068474,2.578816,Temozolomide,Resistant
9638,ACH-000638,NCIH441_LUNG,NCI-H441,LUAD,GDSC2,401,19054580,908460,NCI-H441,SIDM00925,...,1046,Y,0.030016,30.0,8.891349,0.965973,0.077245,2.015987,Temozolomide,Resistant
20149,ACH-000113,OCIAML2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,OCI-AML2,LAML,GDSC2,401,19099928,910947,OCI-AML2,SIDM00446,...,1046,Y,0.030016,30.0,3.647879,0.942202,0.137500,-2.007937,Temozolomide,Sensitive
21621,ACH-000222,ASPC1_PANCREAS,AsPC-1,PAAD,GDSC2,401,19087466,910702,AsPC-1,SIDM00899,...,1046,Y,0.030016,30.0,9.811507,0.969569,0.051331,2.722131,Temozolomide,Resistant
23858,ACH-000308,EFO21_OVARY,EFO-21,OV,GDSC2,401,19101060,911905,EFO-21,SIDM01052,...,1046,Y,0.030016,30.0,9.203067,0.970885,0.055270,2.255204,Temozolomide,Resistant
25708,ACH-000365,SUDHL4_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,SU-DHL-4,DLBC,GDSC2,401,19176070,1331035,SU-DHL-4,SIDM00405,...,1046,Y,0.030016,30.0,3.337333,0.917222,0.242766,-2.246254,Temozolomide,Sensitive
29685,ACH-000648,NCIH28_PLEURA,NCI-H28,MESO,GDSC2,401,19056492,908470,NCI-H28,SIDM00720,...,1046,Y,0.030016,30.0,8.998074,0.967058,0.077005,2.097890,Temozolomide,Resistant
49703,ACH-000759,MDAMB175VII_BREAST,MDA-MB-175-VII,BRCA,GDSC2,401,19041772,908120,MDA-MB-175-VII,SIDM00633,...,1046,Y,0.029297,30.0,9.337230,0.979750,0.067217,2.358164,Temozolomide,Resistant
56009,ACH-000346,JVM3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,JVM-3,DLBC,GDSC2,401,19029322,907270,JVM-3,SIDM01012,...,1046,Y,0.030016,30.0,2.247410,0.785896,0.093411,-3.082679,Temozolomide,Sensitive
59027,ACH-000656,SUDHL8_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,SU-DHL-8,DLBC,GDSC2,401,19176797,1331038,SU-DHL-8,SIDM00423,...,1046,Y,0.030016,30.0,3.203253,0.920825,0.092770,-2.349150,Temozolomide,Sensitive


In [14]:
#from sklearn.preprocessing import MultiLabelBinarizer

### MultiLabelBinarization을 위한 class list 생성
#classes = set(drug_standard_Temozolomide['DRUG_NAME'].to_list())

### MultiLabelBinarization
#mlb = MultiLabelBinarizer()
#mlb.fit([classes])#이건 아무것도 안한데 아래 transform만 있어도 돌아는 가는 것 같아 

#tmp = drug_standard_Temozolomide['DRUG_NAME'].str.split(', ').to_list()
#tmp_result = mlb.transform(tmp)

#df_mlb = pd.DataFrame(tmp_result, columns = mlb.classes_)#약물 one hot code
#df_mlb

##EXPRESSION

In [30]:
#expression data normalization
from sklearn.preprocessing import MinMaxScaler
tzm_id = drug_Tzm['DepMap_ID']
exp_norm1 = pd.merge(tzm_id, exp, left_on = 'DepMap_ID', right_on = 'DepMap_ID')
exp_norm2 = exp_norm1.drop(labels = ['DepMap_ID'], axis=1)

In [ ]:
exp_norm1

In [31]:
scaler = MinMaxScaler()
exp_norm3 = scaler.fit_transform(exp_norm2)
exp_norm3

array([[0.83869328, 0.        , 0.76430731, ..., 1.        , 0.08200757,
        0.49989694],
       [0.95913733, 0.        , 0.69151999, ..., 0.9025058 , 0.03834   ,
        0.80240638],
       [0.00526791, 0.        , 0.35129696, ..., 0.53314369, 0.32902849,
        0.54971526],
       ...,
       [0.03251243, 0.        , 0.        , ..., 0.        , 0.56024602,
        0.22546125],
       [0.03485624, 0.        , 0.73176518, ..., 0.81814575, 0.07489608,
        0.90131892],
       [0.00264699, 0.        , 0.80350789, ..., 0.41389197, 0.08905528,
        0.74954578]])

In [51]:
exp_res = pd.DataFrame(exp_norm3, columns=exp_norm2.columns, index=list(exp_norm1['DepMap_ID']))

In [53]:
exp_res['DepMap_ID'] = exp_res.index

In [ ]:
a = drug_Tzm[['DepMap_ID','measure_of_response']]
exp_res = pd.merge(exp_res, a, left_on='DepMap_ID', right_on='DepMap_ID')
exp_res

In [56]:
exp_res.index = exp_res['DepMap_ID']
exp_res = exp_res.drop(labels = ['DepMap_ID'], axis=1)

In [58]:
exp_res.to_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/cell_Tzm_exp.csv',index=False)#cellline, gdsc랑 매칭되는 Temozolomide, expression 저장

##RPPA

In [61]:
rppa = pd.read_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/CCLE_RPPA_20180123.csv')
b = drug_Tzm[['CELLLINE','measure_of_response']]
rppa_res = pd.merge(b, rppa, left_on='CELLLINE', right_on='Unnamed: 0')

rppa_res.index = rppa_res['CELLLINE']
rppa_res = rppa_res.drop(labels=['CELLLINE'],axis=1)
rppa_res = rppa_res.drop(labels=['Unnamed: 0'],axis=1)

rppa_res

,measure_of_response,14-3-3_beta,14-3-3_epsilon_Caution,14-3-3_zeta,4E-BP1,4E-BP1_pS65,4E-BP1_pT37_T46,4E-BP1_pT70,53BP1,A-Raf_pS299_Caution,...,Tuberin_pT1462,VAV1_Caution,VEGFR2,VHL_Caution,XBP1_Caution,XRCC1_Caution,YAP_Caution,YAP_pS127_Caution,YB-1,YB-1_pS102
CELLLINE,,,,,,,,,,,,,,,,,,,,,
COLO678_LARGE_INTESTINE,Resistant,0.074803,-0.189680,0.097233,-0.001196,0.402218,0.536360,0.097793,-1.068015,0.221810,...,-0.100349,-0.509252,1.988518,0.161715,-0.647749,0.008009,0.401648,1.639161,-0.380819,-0.199896
NCIH441_LUNG,Resistant,0.300705,-0.089912,0.477705,-0.077647,-0.125315,-0.494754,-0.194617,-0.210285,0.164503,...,-0.241770,3.826751,0.268184,0.816209,0.047767,-0.242476,-0.076736,0.065670,-0.002194,-0.021275
OCIAML2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Sensitive,0.066119,0.018159,1.203209,0.849625,0.186713,-1.075618,-0.105340,-0.100726,-0.121524,...,0.505938,4.504354,-0.499097,0.444579,-0.047446,0.191960,-0.956440,-2.654187,0.545946,-1.099643
ASPC1_PANCREAS,Resistant,0.404863,-0.200637,0.737979,-0.153710,-0.282240,-0.099987,-0.408458,-0.434957,-0.031708,...,0.477504,-0.687980,2.028262,0.407897,0.000439,-0.446418,-0.003445,0.413738,-0.248885,0.427690
EFO21_OVARY,Resistant,0.153023,-0.111463,-0.926636,-0.508999,-0.242104,-0.901306,-0.033525,0.812741,-0.011415,...,-0.176444,0.201658,-0.920156,3.389482,0.131191,0.069679,-0.140510,-0.103485,0.206828,-0.192376
SUDHL4_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Sensitive,-0.017900,0.144190,-0.235443,-0.045999,-0.240608,-0.425730,0.211159,-0.771094,0.057591,...,0.174426,5.363528,-2.838270,-0.588125,0.357554,-0.038817,-1.121239,-2.253224,0.017317,-0.789465
NCIH28_PLEURA,Resistant,-0.018499,0.049884,-0.139244,-0.609266,0.457305,0.273207,0.006909,0.352098,0.405747,...,-0.011475,-0.170293,0.244859,-1.228024,0.161052,-0.577317,0.525533,0.262434,-0.530495,0.434713
MDAMB175VII_BREAST,Resistant,0.037009,-0.017792,0.308290,-1.142862,-0.052724,-0.254600,-0.640859,-0.333303,0.995723,...,0.964484,0.142075,1.128642,4.031608,-0.291824,-0.094397,-0.425804,-1.429306,-0.319926,-0.128125
JVM3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Sensitive,0.070827,0.101149,-0.347467,-0.098456,-0.574636,-0.722362,-0.056910,-0.052698,0.057895,...,-0.041886,5.704526,-1.377233,-0.991332,0.433173,0.150821,-1.118629,-2.524594,-0.051596,-0.174199


In [ ]:
rppa_res

In [ ]:
import matplotlib.pyplot as plt
X = rppa_res.drop(['measure_of_response'], axis=1)
plt.hist(X)

In [64]:
rppa_res.to_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/cell_Tzm_exp_rppa.csv',index=False)#cellline, gdsc랑 매칭되는 Temozolomide 저장